In [1]:
from myur import MyUR3e
import time
robot = MyUR3e()

In [2]:
STACK1 = [0.15,0.3] # stack1 x,y
STACK2 = [0.25,0.3] # stack2 x,y
ACTIVE = [0.23,0,0,0] # transfer height, orientation
DROP = [] # relative drop trajectory, 15cm max
for i in range(15):
    DROP.append([0,0,-0.01,0,0,0])

In [3]:
# function to detect block
def detect():
    # close gripper and start drop
    robot.move_gripper(100,wait=False)
    robot.move_global_r(DROP,time=7,wait=False,interp="linear")

    # get comparative z force and monitor in loop
    comp_force = robot.read_force()[2]
    z_force = robot.read_force()[2]
    while abs(comp_force-z_force) < 1:
        z_force = robot.read_force()[2]
        time.sleep(0.05)

    # stop robot when object detected
    print("Object detected")
    robot.stop()

# function to grab block
def grab():
    robot.move_global_r([[0,0,0.02,0,0,0]],time=0.3,wait=True)
    robot.move_gripper(0)
    robot.move_global_r([[0,0,-0.03,0,0,0]],time=0.3,wait=True)
    robot.move_gripper(100)

# function to drop block
def drop():
    robot.move_gripper(0,wait=False)
    robot.move_global_r([[0,0,0.03,0,0,0]],time=0.3,wait=True)
    robot.move_gripper(100,wait=False)

# reset pose
def reset_pos():
    robot.move_gripper(0,wait=False)
    robot.move_global([STACK2+ACTIVE],time=3)
    robot.move_gripper(100)

In [ ]:
reset_pos()
while True: # forever move 3 blocks bewtween stack 1 & 2
    for i in range(3):
        robot.move_global([STACK2+ACTIVE],time=1)
        detect()
        grab()
        robot.move_global([STACK2+ACTIVE,STACK1+ACTIVE],time=1)
        detect()
        drop()
        robot.move_global([STACK1+ACTIVE],time=1)
    for i in range(3):
        robot.move_global([STACK1+ACTIVE],time=1)
        detect()
        grab()
        robot.move_global([STACK1+ACTIVE,STACK2+ACTIVE],time=1)
        detect()
        drop()
        robot.move_global([STACK2+ACTIVE],time=1)